$\Huge Code$ $\hspace{0.1cm}$ $\Huge to$ $\hspace{0.1cm}$ $\Huge simulate$ $\hspace{0.1cm}$ $\Huge kSZ$ $\hspace{0.1cm}$ $\Huge maps$ $\hspace{0.1cm}$ $\Huge at$ $\hspace{0.1cm}$ $\Huge differents$ $\hspace{0.1cm}$ $\Huge frequencies$ $\hspace{0.1cm}$ $\Huge from$ $\hspace{0.1cm}$ $\Huge a$ $\hspace{0.1cm}$ $\Huge "y_{kSZ}"$ $\hspace{0.1cm}$ $\Huge map$ $\Huge :$ 

# TO DO :

# Modules : 

In [63]:
import healpy as hp
import matplotlib.pyplot as plt 
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
from astropy.cosmology import FlatLambdaCDM
from astropy import constants as cst
import numpy as np 

# Physics : 

$T_{CMB}$ is the temperature of the CMB. To get the proper kSZ map we can use for our study, we need to divide each kSZ map at a given frequency by a given vector, which will turn the Jysr$^{-1}$ map into a $\frac{\Delta T}{T_{CMB}}$ map which is dimensionless. To get a map in $\mu K$, we need to multiply by $T_{CMB}\times 10^{6}$

In [64]:
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Tcmb0=2.7255)
T_CMB = cosmo.Tcmb0.si.value
k_B = cst.k_B.value
h = cst.h.value
c = cst.c.value

# Functions : 

## Frequency conversion factor :

We have a each frequency : $\Delta I=-I_{0}y_{kSZ}\frac{x^{4}e^{x}}{(e^{x}-1)^{2}}$

In [65]:
def DI_kSZ(x):

    """
    Function which compute the DI conversion factor.

    Parameters
    ----------
    
    x : str
        Path were the data of the maps are stored and we the cutout are going to be stored. 
    y : str
        Name of the data file. 
        
    Returns
    -------
    str
        Tell us where the function stored the datas and images.

    """
    
    I_0 = 2*(k_B*T_CMB)**3/(h*c)**2*1e26
    x_nu = (h*x)/(k_B*T_CMB)
    Delta_I = (-I_0*x_nu**4*np.exp(x_nu))/((np.exp(x_nu)-1)**2)
    
    print("Delta I as been computed ")
    
    return   Delta_I

In [66]:
def mixing_vector(dic_freq): 
    
    freq = np.arange(0,1000)*10**9  
    Delta_I = DI_kSZ(freq)
    mix_vect = []
    
    for i in range(0,len(dic_freq)):
        mix_vect.append(Delta_I[dic_freq[i]])
    print('The mixing vector is : ',mix_vect)
    
    plt.xticks(fontsize=28)
    plt.yticks(fontsize=28)
    plt.rcParams['figure.figsize'] = [15, 10]
    plt.ylabel('$\Delta I (MJysr^{-1})$',fontsize=28)
    plt.xlabel('$Frequency$ $(GHz)$',fontsize=28)
    plt.title('')
    plt.hlines(y=0.,xmin=0,xmax=10000, colors='k', linestyles='solid') 
    plt.plot(freq,np.array(Delta_I))
    plt.show()

    return mix_vect

## Construct kSZ spectral shape and conversion factor : 

The equation to convert between a change in intensity ($\Delta I$) and a change of Temperature ($\Delta T_{CMB}$) is : 

$\frac{\Delta I}{\Delta T_{CMB}}=\frac{I_0}{T_{CMB}}\frac{x^{4}e^{x}}{(e^{x}-1)^{2}}$ or $\Delta I=-I_{0}y_{kSZ}\frac{x^{4}e^{x}}{(e^{x}-1)^{2}}$ with $y_{kSZ}=1$

Then to go from a map in Jysr$^{-1}$ to a map in $\mu K$ we need to multiply the maps by the INVERSE of : 

$\Delta T_{CMB} = \frac{T_{CMB}\times 10^{6}\Delta I}{I_0\times 10^{26}}\frac{(e^{x}-1)^{2}}{x^{4}e^{x}}$

$I_0\times 10^{20}=270.33$ is converted from [MJy/sr] to [Jy/sr] by multiplying by $10^{6}$. To get the maps in $\mu K$, we multiply $T_{CMB}$ [K] by $10^{6}$ also. 

In [67]:
def DT_kSZ(x,y):

    """
    Function which compute the kSZ spectral shape and/or the conversion.

    Parameters
    ----------
    
    x : str
        Path were the data of the maps are stored and we the cutout are going to be stored. 
    y : str
        Name of the data file. 
        
    Returns
    -------
    str
        Tell us where the function stored the datas and images.

    """
    
    I_0 = 2*(k_B*T_CMB)**3/(h*c)**2*1e26
    x_nu = (h*x)/(k_B*T_CMB)
    Delta_T = ((np.exp(x_nu)-1)**2)/(I_0*x_nu**4*np.exp(x_nu))
    
    print("Delta T as been computed ")
    
    return   Delta_T

In [68]:
def conv_vector(dic_freq): 
    
    freq = np.arange(0,1000)*10**9  
    Delta_T = DT_kSZ(freq,1)
    conv_vect = []
    
    for i in range(0,len(dic_freq)):
        conv_vect.append(1/Delta_T[dic_freq[i]])
    print('The conversion vector is : ',conv_vect)
    
    plt.xticks(fontsize=28)
    plt.yticks(fontsize=28)
    plt.rcParams['figure.figsize'] = [15, 10]
    plt.ylabel('$\Delta I (MJysr^{-1})$',fontsize=28)
    plt.xlabel('$Frequency$ $(GHz)$',fontsize=28)
    plt.title('')
    plt.hlines(y=0.,xmin=0,xmax=10000, colors='k', linestyles='solid') 
    plt.plot(freq,np.array(Delta_T))
    plt.show()

    return conv_vect

## Convert units : 

In [69]:
def convert_units(freq, values, cmb2mjy=False, mjy2cmb=False, rj2mjy=False, mjy2rj=False, cmb2rj=False,
                  rj2cmb=False):
    
    '''
    Convert observed signal at given frequencies to different units. 
    Parameters
    ----------
    freq: float or float array
        Frequency in Hz.
    values: float or float array
        Measured signal.
    cmb2mjy: bool, optional
        If True, the input is assumed to be K_CMB, the output will be MJy/sr.
        Default: False
    mjy2cmb: bool, optional
        If True, the input is assumed to be MJy/sr, the output will be K_CMB.
        Default: False
    rj2mjy: bool, optional
    If True, the input is assumed to be K_RJ, the output will be MJy/sr.
    Default: False
    mjy2rj: bool, optional
        If True, the input is assumed to be MJy/sr, the output will be K_RJ.
        Default: False
    cmb2rj: bool, optional
        If True, the input is assumed to be K_CMB, the output will be K_RJ.
        Default: False
    rj2cmb: bool, optional
        If True, the input is assumed to be K_RJ, the output will be K_CMB.
        Default: False
    Returns
    -------
    converted_signal: float or float array
        Converted signal
    '''

    x = h * freq / k_B / T_CMB
    
    if cmb2mjy is True:
        conversion = 1e20 * 2*k_B**3.*T_CMB**2. / (h * c)**2. * x**4. * np.exp(x) / (np.exp(x)-1)**2.
    elif mjy2cmb is True:
        conversion = 1/(1e20 * 2*k_B**3.*T_CMB**2. / (h * c)**2. * x**4. * np.exp(x) / (np.exp(x)-1)**2.)
    elif rj2mjy is True:
        conversion = 1e20 * 2*freq**2.*k_B/c**2.
    elif mjy2rj is True:
        conversion = 1/(1e20 * 2*freq**2.*k_B/c**2.)
    elif cmb2rj is True:
        conversion = (k_B*T_CMB/h)**2. * x**4. * np.exp(x) / (np.exp(x)-1)**2. / freq**2.
    elif rj2cmb is True:
        conversion = 1/((k_B*T_CMB/h)**2. * x**4. * np.exp(x) / (np.exp(x)-1)**2. / freq**2.)        
    else:
        print("Not sure which units are given and what should be returned.")

    converted_signal = conversion * values

    return(converted_signal)

## Multiply a map by a given vector : 

Now that we have the mixing vector for the frequencies used in our data, we can multiply the kSZ map by this vector, which will give us the "$y_{kSZ}$". 

In [70]:
def udgrade_NSIDE(data_path,pictures_path,file_name,title_omap,title_nmap,map_unit,nside,name_file_udmap):
    
    """
    Function which upgrade or degrade the NSIDE of an HEALPix map. 

    Parameters
    ----------
    
    data_path : str
        Path were the data of the maps are stored. 
    pictures_path : str 
        Path where we are going to save the pictures. 
    file_name : str 
        Name of the FITS file containing the map we want to upgrade or degrade. 
    title_omap : str 
        Title displayed on the original map. 
    title_nmap : str
        Title displayed on the upgraded or degraded map. 
    map_unit : str 
        Unit of the map we want to upgrade or degrade, will appear on the HEALPix respresentation.
    nside : int 
        Value of the new resolution we want to apply to the healpix map to upgrade or degrade it.
    name_file_udmap : str 
        Name of the FITS file and image name under which the updgraded or degraded map is going to be saved.  
        
    Returns
    -------
    str
        Tell us where the function stored the images. 

    """
    
    #Open the original map and display it : 
    maps = hp.read_map(data_path + file_name)
    hp.mollview(map=maps,coord=None, nest=False, title=title_omap, unit=map_unit, norm='hist')
    
    #Upgarde or degrade the map : 
    ud_map = hp.pixelfunc.ud_grade(map_in=maps, nside_out=nside, pess=False, order_in='RING', order_out=None, power=None, dtype=None)
    
    #Write the map as a FITS file and display it : 
    A = hp.write_map(data_path + name_file_udmap + '.fits',ud_map,overwrite=True)
    hp.mollview(map=ud_map,coord=None, nest=False, title=title_nmap, unit=map_unit, norm='hist')
    plt.savefig(pictures_path + name_file_udmap  + '.png')
    plt.show()
    
    return A

In [71]:
def mapxvalue(data_path,file_in,multiplier,simu,map_unit,pictures_path,dic_freq,factor,name_object,freq_unit,
                  data_save,convert,multiply): 

    """
    Function which take a Halpix map and multiply it by a vector. 

    Parameters
    ----------
    
    data_path : str
        Path were the data of the maps are stored and we the cutout are going to be stored. 
    file_in : str
        Name of the data file. 
    multiplier : list
        Vector containing the different values you want to divide the map with. 
    data_origin : str
        Origin of the data used here, for exemple Planck AllSky maps. This is going to be use for figure titles and 
        names of files. 
    maps_unit : str 
        Units of the maps that are displayed, for exemple K_CMB, MJy/sr. This is going to be use for figure titles and
        names of files.  
    pictures_path : str
        Path where we are going to save the pictures.   
    dic_freq : dict
        Dictonary containing all the frequencies we want to deal with for those peculiar data. 
    factor : float 
        Number by which we can multiply the map getting out, the map that is going to be saved.        
    name_object : str
        Type of Healpix map. FOR EXEMPLE : CMB, tSZ, kSZ. 
    freq_unit : str
        Units of the frequencies, for exemple GHz. This is going to be use for figure titles and names of files. 
    data_save : str
        path under which the new maps will be saved.
        
    Returns
    -------
    str
        Tell us where the function stored the datas and images.

    """
    
    # Open the datas :    

   
    
    if multiply == True: 
        
        maps = file_in
        
        #Frequency change : 
        maps_freq = (maps*multiplier) 

        #Dislay and save tha map : 
        print(name_object + ' map at the frequency ' + str(dic_freq) + freq_unit + ' :')
        hp.mollview(map=maps_freq, coord=None, nest=False,title='kSZ at ' + str(dic_freq) + 'GHz from ' + simu, unit=map_unit, 
                    norm='hist',xsize=2000) #Display a read FITS file with Healpy and save it
        plt.savefig(pictures_path + 'kSZ_'+ str(dic_freq) + '_' + simu + '.png') #Save the figure        
        plt.show()   


        #Save file : 
        hp.write_map(data_save + 'kSZ_'+ str(dic_freq[i]) + '_' + simu + '.fits', maps_freq,nest=False,dtype=np.float32,
                     overwrite=True)
        print('Maps is saved under the name : ' + data_save + 'kSZ_'+ str(dic_freq[i]) + '_' + simu + '.fits')


        print("Map has been multiplied by the mixing vector, Images are saved in " + data_save)
        
    
    maps = hp.read_map(data_path+file_in)
        
    for i in range(len(dic_freq)):
    
        if convert == True: 
            
            #Convert the map :
            maps_freq = (maps / multiplier[i])*T_CMB*10**6*factor
            
            #Dislay and save tha map : 
            print(name_object + ' map at the frequency ' + str(dic_freq[i]) + freq_unit + ' :')
            hp.mollview(map=maps_freq, coord=None, nest=False,title='ykSZ from ' + simu, unit=map_unit, 
                    norm='hist',xsize=2000) #Display a read FITS file with Healpy and save it
            plt.savefig(pictures_path + 'ykSZ_' + simu + '.png') #Save the figure        
            plt.show()   


            #Save file : 
            hp.write_map(data_save + 'ykSZ_' + simu + '.fits', maps_freq,nest=False,dtype=np.float32,
                     overwrite=True)
            print('Maps is saved under the name : ' + data_save + 'ykSZ_' + simu + '.fits')


            print("Map has been multiplied by the mixing vector, Images are saved in " + data_save)
    
        else:
            
            #Frequency change : 
            maps_freq = (maps*multiplier[i]) 

            #Dislay and save tha map : 
            print(name_object + ' map at the frequency ' + str(dic_freq[i]) + freq_unit + ' :')
            hp.mollview(map=maps_freq, coord=None, nest=False,title='kSZ at ' + str(dic_freq[i]) + 'GHz from ' + simu, unit=map_unit, 
                    norm='hist',xsize=2000) #Display a read FITS file with Healpy and save it
            plt.savefig(pictures_path + 'kSZ_'+ str(dic_freq[i]) + '_' + simu + '.png') #Save the figure        
            plt.show()   


            #Save file : 
            hp.write_map(data_save + 'kSZ_'+ str(dic_freq[i]) + '_' + simu + '.fits', maps_freq,nest=False,dtype=np.float32,
                     overwrite=True)
            print('Maps is saved under the name : ' + data_save + 'kSZ_'+ str(dic_freq[i]) + '_' + simu + '.fits')


            print("Map has been multiplied by the mixing vector, Images are saved in " + data_save)

        
    return 

In [89]:
def simulate_kSZ(simu,dic_freq,unit_out):
    
    if simu == 'SO':
        
        #Fixed datas : 
        data_path='/vol/arc3/data1/sz/SO_sky_model/CMB_SZ_maps/'
        data_save = '/vol/arc3/data1/sz/CCATp_sky_model/workspace_maude/SO/'
        pictures_path = '/vol/arc3/data1/sz/SO_sky_model/pictures/'
        file_in = data_path + '148_ksz_healpix_nopell_Nside4096_DeltaT_uK.fits'

        name_object = 'tSZ'
        freq_unit='GHz'
    
        if unit_out == 'mK': 
        
            # Open, display and save the y_kSZ map : 
            ykSZ_map = hp.read_map(file_in,nest=False,partial=False) #Read a FITS file with Healpy
                    
            print('Here is the "y_kSZ" map from '+ simu + ' :')
            hp.mollview(map=ykSZ_map, coord=None, nest=False,title='$y_{kSZ}$ $map$ $from$ '+ simu, unit='$\mu K$',
                        norm='hist', xsize=2000,return_projected_map=True) #Display a read FITS file with Healpy and save it
            plt.savefig(pictures_path + 'ykSZ_'+simu+'.png')
            plt.show()

            #Save file : 
            hp.write_map(data_save + 'ykSZ_' + simu + '.fits', ykSZ_map,nest=False,dtype=np.float32,overwrite=True)

            #Feedback operator : 
            print('Maps is saved under the name : ' + data_save + 'Compton-y_' + data_origin + '.fits')       
    
    
    if simu == 'Sehgal' : 
        
        #Fixed datas :         
        data_path='/vol/arc3/data1/sz/Sehgal/'
        data_save = '/vol/arc3/data1/sz/CCATp_sky_model/workspace_maude/Sehgal/'
        pictures_path = '/vol/arc3/data1/sz/Sehgal/pictures/'
        file_in = '030_ksz_healpix.fits'
        name_object = 'y_kSZ'
        
    
    print("Map has been multiplied by the mixing vector, Images are saved in " + data_save)
        
    if unit_out == 'mK':
      
        #Produce y-kSZ : 
            
        # Open the datas :    
        maps = hp.read_map(data_path+file_in)
        
        #Convert the map :
        maps_freq = (maps / conv_vector({0:30}))*T_CMB*10**6

        #Dislay and save tha map : 
        print(name_object + ' :')
        hp.mollview(map=maps_freq, coord=None, nest=False,title='ykSZ from ' + simu, unit='$\mu K$', 
                    norm='hist',xsize=2000) #Display a read FITS file with Healpy and save it
        plt.savefig(pictures_path + 'ykSZ_' + simu + '.png') #Save the figure        
        plt.show()   
         
        #Save file : 
        hp.write_map(data_save + 'ykSZ_' + simu + '.fits', maps_freq,nest=False,dtype=np.float32,
                     overwrite=True)
        print('Maps is saved under the name : ' + data_save + 'ykSZ_' + simu + '.fits')
        
            
        #Produce kSZ at different frequencies : 
        multi = mixing_vector(dic_freq)

        mapxvalue(data_path=data_path,file_in=file_in,multiplier=multi,simu=simu,map_unit='$\mu K$',
                    pictures_path=pictures_path,dic_freq=dic_freq,factor=1,name_object='ykSZ',freq_unit='GHz',
                    data_save=data_save,convert=False,multiply=False)
        
    if unit_out=='Jysr': 
            
        freq = np.arange(0,1000)*10**9  
        Delta_I = DI_kSZ(freq)
        
        for i in range(len(dic_freq)):

            mix_vect = (Delta_I[dic_freq[i]])
            print(mix_vect)

            A = mapxvalue(data_path=data_path,file_in=file_in,multiplier=mix_vect,simu=simu,map_unit='$\mu K$',
                              pictures_path=pictures_path,dic_freq=dic_freq[i],factor=1,name_object='kSZ',freq_unit='GHz',
                              data_save=data_save,convert=False,multiply=True)
                
            mapxvalue(data_path=data_path,file_in=A,multiplier=mix_vect,simu=simu,map_unit='$Jysr$',
                          pictures_path=pictures_path,dic_freq=dic_freq,factor=1/(T_CMB*10**6),name_object='ykSZ',freq_unit='GHz',
                          data_save=data_save,convert=False)
        
            

# Launch : 

In [90]:
simulate_kSZ(simu='Sehgal',dic_freq={0:30,1:90,2:148,3:219,4:277,5:350},unit_out='Jysr') 

Map has been multiplied by the mixing vector, Images are saved in /vol/arc3/data1/sz/CCATp_sky_model/workspace_maude/Sehgal/
Delta I as been computed 
-73635005.0206906


/users/mcharmetant/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


TypeError: can't multiply sequence by non-int of type 'numpy.float64'